# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [34]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [35]:
df = pd.read_excel(r'C:\Users\user\матрица\Экс3.xlsx')
df.head()

,Вид происшествия,Аварии,Инциденты,Предпосылки
0,Аварии,1.000000,7.000000,9
1,Инциденты,0.142857,1.000000,6
2,Предпосылки к инцидентам,0.111111,0.166667,1


In [36]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (3, 4)


In [37]:
# типы столбцов 
df.dtypes

Вид происшествия     object
Аварии              float64
Инциденты           float64
Предпосылки           int64
dtype: object

In [38]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Вид происшествия') 
print(df)

     Аварии  Инциденты  Предпосылки
0  1.000000   7.000000            9
1  0.142857   1.000000            6
2  0.111111   0.166667            1


In [39]:
# типы столбцов 
df.dtypes

Аварии         float64
Инциденты      float64
Предпосылки      int64
dtype: object

In [40]:
# также меням тип данных Предпосылки, Инциденты, Аварии
df['Предпосылки'] = df['Предпосылки'].astype(float)
df['Инциденты'] = df['Инциденты'].astype(float)
df['Аварии'] = df['Аварии'].astype(float)

In [41]:
df.loc[:, df.columns[df.dtypes == 'float64']].head()

,Аварии,Инциденты,Предпосылки
0,1.000000,7.000000,9.0
1,0.142857,1.000000,6.0
2,0.111111,0.166667,1.0


In [42]:
DF_1=df[:].copy()

In [43]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Аварии          1.253968
Инциденты       8.166667
Предпосылки    16.000000
dtype: float64

In [44]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Аварии'] = df['Аварии'].astype(float).div(df['Аварии'].sum())
df['new_Инциденты'] = df['Инциденты'].astype(float).div(df['Инциденты'].sum())
df['new_Предпосылки'] = df['Предпосылки'].astype(float).div(df['Предпосылки'].sum())

In [45]:
df.head()

,Аварии,Инциденты,Предпосылки,new_Аварии,new_Инциденты,new_Предпосылки
0,1.000000,7.000000,9.0,0.797468,0.857143,0.5625
1,0.142857,1.000000,6.0,0.113924,0.122449,0.3750
2,0.111111,0.166667,1.0,0.088608,0.020408,0.0625


In [46]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Аварии', 'new_Инциденты', 'new_Предпосылки']]
df_new

,new_Аварии,new_Инциденты,new_Предпосылки
0,0.797468,0.857143,0.5625
1,0.113924,0.122449,0.3750
2,0.088608,0.020408,0.0625


In [47]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Аварии,new_Инциденты,new_Предпосылки,mean
0,0.797468,0.857143,0.5625,0.739037
1,0.113924,0.122449,0.3750,0.203791
2,0.088608,0.020408,0.0625,0.057172


In [48]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

0.9999999999999999

In [49]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new4 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

In [50]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new4)
print(res1)
res2 = np.dot(df_new2,df_new4)
print(res2)
res3 = np.dot(df_new3,df_new4)
print(res3)

2.680121415731307
0.652399250876138
0.17325231777746603


In [51]:
sum_new = res1 + res2 + res3 
print(sum_new)

3.505772984384911


In [52]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 3)/(3-1)
print(CI)
RI = (1.98*(3-2))/3
print(RI)
CR = CI/RI
print(CR)

0.2528864921924554
0.66
0.3831613518067506


In [53]:
# Получилось, что значение CR=0.38, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами